required tmc data : TMC_Identification.csv

In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd
import osm2gmns as og

osm2gmns, version 0.4.2


In [2]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def Create_Boundary(node_csv):
    
    minx = node_csv['x_coord'].min()
    miny = node_csv['y_coord'].min()
    maxx = node_csv['x_coord'].max()
    maxy = node_csv['y_coord'].max()
    bbox = [minx,miny,maxx,maxy]
    return bbox

In [4]:
tmc_path = '/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/TMC'
gmns_path = '/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/GMNS'


osm_path = 'OSM/consolidated'
net = og.getNetFromOSMFile('OSM/tmc_VA.osm',network_type=('auto'), default_lanes=True, default_speed=True)
og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=osm_path)

In [11]:
create_folder(gmns_path)
print('reading tmc data...')

tmc = pd.read_csv(tmc_path + os.sep + 'TMC_Identification_VA.csv')
# tmc = tmc.sort_values(by=['road_order'])

'''build node.csv'''
print('converting tmc data into gmns format...')


node_csv = pd.DataFrame()
node_csv['name'] = None
node_csv['x_coord'] = None
node_csv['y_coord'] = None
node_csv['z_coord'] = None
node_csv['node_type'] = None
node_csv['ctrl_type'] = None
node_csv['zone_id'] = None
node_csv['parent_node_id'] = None
node_csv['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_csv.index.name = 'node_id'

node_csv.index += 100000001 #index from 0

node_csv.to_csv(gmns_path + os.sep + '/node_tmc.csv')

reading tmc data...
converting tmc data into gmns format...


In [12]:
'''build link.csv'''
link_csv = pd.DataFrame()
link_csv['name'] = None
link_csv['from_node_id'] = None
link_csv['to_node_id'] = None
link_csv['directed'] = None
link_csv['geometry_id'] = None
link_csv['geometry'] = None
link_csv['dir_flag'] = None
link_csv['parent_link_id'] = None
link_csv['length'] = None
link_csv['grade'] = None
link_csv['facility_type'] = None
link_csv['capacity'] = None
link_csv['free_speed'] = None
link_csv['lanes'] = None

for i in range(0,len(tmc)):
    link_csv = link_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'from_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'start_longitude']) & (node_csv['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                    'to_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'end_longitude']) & (node_csv['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                    'directed': 1,\
                                    'geometry_id': None,\
                                    'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                        tmc.loc[i,'end_longitude'].astype(str) + tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                    'dir_flag': 1,\
                                    'parent_link_id': None,\
                                    'length': tmc.loc[i,'miles'],\
                                    'grade': None,\
                                    'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                    'capacity':None,\
                                    'free_speed':None,\
                                    'lanes': None}, ignore_index=True)
link_csv.index.name = 'link_id'
link_csv.index += 100000001


link_csv.to_csv(gmns_path + os.sep + '/link_tmc.csv')


In [13]:
link_csv

,name,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,
100000001,110N05711,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.370939.0141)",1,None,0.038620,None,None,None,None,None
100000002,110-05710,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.368439.0127)",1,None,0.164895,None,None,None,None,None
100000003,110N05710,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.360539.0086)",1,None,0.512158,None,None,None,None,None
100000004,110-05709,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.350239.0028)",1,None,0.682589,None,None,None,None,None
100000005,110-05708,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.345539.0005)",1,None,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.3539.003)",1,None,0.254799,None,None,None,None,None
100000059,110+05710,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.360939.009)",1,None,0.718593,None,None,None,None,None
100000060,110P05710,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.367339.0125)",1,None,0.421999,None,None,None,None,None


In [14]:
tmc

,tmc,road,direction,intersection,state,county,zip,start_latitude,start_longitude,end_latitude,end_longitude,miles,road_order
0,110N05711,VA-7,EASTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0144,-77.3715,39.0141,-77.3709,0.038620,38.0
1,110-05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0141,-77.3709,39.0127,-77.3684,0.164895,39.0
2,110N05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0127,-77.3684,39.0086,-77.3605,0.512158,40.0
3,110-05709,VA-7,EASTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0086,-77.3605,39.0028,-77.3502,0.682589,41.0
4,110-05708,VA-7,EASTBOUND,VA-193/GEORGETOWN PIKE,VA,FAIRFAX COUNTY,20170,39.0028,-77.3502,39.0005,-77.3455,0.296731,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,110+05709,VA-7,WESTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0010,-77.3460,39.0030,-77.3500,0.254799,45.0
58,110+05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0030,-77.3500,39.0090,-77.3609,0.718593,46.0
59,110P05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0090,-77.3609,39.0125,-77.3673,0.421999,47.0
60,110+05711,VA-7,WESTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0125,-77.3673,39.0143,-77.3707,0.220479,48.0


In [15]:
'''build link.csv'''
node_csv


,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,110N05711,-77.3715,39.0144,None,tmc_start,None,None,None,POINT (-77.3715 39.0144)
100000002,110-05710,-77.3709,39.0141,None,tmc_start,None,None,None,POINT (-77.3709 39.0141)
100000003,110N05710,-77.3684,39.0127,None,tmc_start,None,None,None,POINT (-77.3684 39.0127)
100000004,110-05709,-77.3605,39.0086,None,tmc_start,None,None,None,POINT (-77.3605 39.0086)
100000005,110-05708,-77.3502,39.0028,None,tmc_start,None,None,None,POINT (-77.3502 39.0028)
...,...,...,...,...,...,...,...,...,...
100000060,110+05710,-77.3500,39.0030,None,tmc_start,None,None,None,POINT (-77.35 39.003)
100000061,110P05710,-77.3609,39.0090,None,tmc_start,None,None,None,POINT (-77.3609 39.009)
100000062,110+05711,-77.3673,39.0125,None,tmc_start,None,None,None,POINT (-77.3673 39.0125)


In [16]:
import os
import math
import heapq
import numpy as np
import pandas as pd


# WGS84 transfer coordinate system to distance: meter
def LLs2Dist(lon1, lat1, lon2, lat2):
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    dist = R * c * 1000
    return dist


def createConnector(dataList,from_node_id, to_node_id):
    link = []
    from_node_id_x = dataList[from_node_id]['x_coord']
    from_node_id_y = dataList[from_node_id]['y_coord']
    to_node_id_x = dataList[to_node_id]['x_coord']
    to_node_id_y = dataList[to_node_id]['y_coord']
    length = LLs2Dist(from_node_id_x,from_node_id_y,to_node_id_x,to_node_id_y)
    geometry = 'LINESTRING (' + str(from_node_id_x)+' '+str(from_node_id_y)+', '+str(to_node_id_x)+' '+str(to_node_id_y)+')'
        
    link = [from_node_id, to_node_id, length, geometry]
    return link




print('creating connector between osm network and gtfs data...')

node_tmc = pd.read_csv(gmns_path + os.sep +'/node_tmc.csv',low_memory=False) # transit node
node_road = pd.read_csv(osm_path + os.sep +'/node.csv', encoding='gbk',low_memory=False)

node_combine = pd.DataFrame()
node_combine['node_id']= node_road['node_id'].tolist() + node_tmc['node_id'].tolist()
node_combine['name']= node_road['name'].tolist() + node_tmc['name'].tolist()
node_combine['x_coord']= node_road['x_coord'].tolist() + node_tmc['x_coord'].tolist()
node_combine['y_coord'] = node_road['y_coord'].tolist() + node_tmc['y_coord'].tolist()
node_combine['node_type']= node_road['node_type'].tolist() + node_tmc['node_type'].tolist()


node_combine.to_csv(gmns_path + os.sep +'/node_combine.csv', index=False)
    
link_road = pd.read_csv(osm_path + os.sep +'/link.csv', encoding='gbk',low_memory=False)
link_road = link_road.drop(['link_id', 'osm_way_id', 'from_biway'], axis=1)
    
dataList_tmc = {}
gp = node_tmc.groupby('node_id')
for key, form in gp:
    dataList_tmc[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

dataList_road = {}
gp = node_road.groupby('node_id')
for key, form in gp:
    dataList_road[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

coord_list = []
for key in dataList_road.keys(): 
    coord_list.append((dataList_road[key]['x_coord'],dataList_road[key]['y_coord']))
coord_array = np.array(coord_list)


dataList = {}  #road node+tmc node
gp = node_combine.groupby('node_id')
for key, form in gp:
    dataList[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

# print('building connector...')
link_list = []

for key in dataList_tmc.keys(): 
    coord = np.array((dataList_tmc[key]['x_coord'],dataList_tmc[key]['y_coord']))
    coord_diff = coord_array - coord
    coord_diff_square = np.power(coord_diff,2)
    coord_diff_sum_square = coord_diff_square.sum(axis=1)
    distance = np.sqrt(coord_diff_sum_square)  #distance array between one tmc node to all road node 
    
    count = 1
    while (count):
        idx_temp = heapq.nsmallest(count, distance.tolist())
        idx = np.where(distance == idx_temp[count-1])  #index of the nearest road node
        active_node = node_road['node_id'].iloc[idx[0]]  #id of the nearest road node
        if ((active_node[idx[0][0]] in link_road['from_node_id'].tolist()) and (active_node[idx[0][0]] in link_road['to_node_id'].tolist())): #check the road node is the from/to node of road link 
            active_link1 = createConnector(dataList,active_node[idx[0][0]], key) #create link between the nearest road node and the selected tmc node
            active_link2= createConnector(dataList,key, active_node[idx[0][0]])
            link_list.append(active_link1)
            link_list.append(active_link2)
            break
        else:
            count += 1


connector_csv = pd.DataFrame()
connector_csv = pd.DataFrame(link_list, columns=['from_node_id','to_node_id','length','geometry']).drop_duplicates()    


connector_csv['name'] = None
connector_csv['link_type'] = None
connector_csv['link_type_name'] = 'connector'
connector_csv['dir_flag'] = 1
connector_csv['lanes'] = 1
connector_csv['free_speed'] = None
connector_csv['capacity'] = None
connector_csv['allowed_uses'] = None



combined_link = pd.concat([link_road,connector_csv], axis=0, ignore_index=True)

combined_link.index.name = 'link_id'
combined_link.index += 1

combined_link.to_csv(gmns_path + os.sep +'/link_osm_connector.csv')



creating connector between osm network and gtfs data...


In [17]:
combined_link

,name,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses
link_id,,,,,,,,,,,,
1,NaN,4,5,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1878249 38.9744185, -77.187663...",auto
2,NaN,5,4,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1869767 38.9755412, -77.187013...",auto
3,NaN,6,7,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.194886 38.9751513, -77.1950799...",auto
4,NaN,7,6,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196016...",auto
5,NaN,7,8,1,85.661050,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196222...",auto
...,...,...,...,...,...,...,...,...,...,...,...,...
30827,None,100000062,12388,1,84.064216,1,None,None,connector,None,"LINESTRING (-77.3673 39.0125, -77.3680805 39.0...",None
30828,None,10238,100000063,1,7.938151,1,None,None,connector,None,"LINESTRING (-77.3707839 39.014329100000005, -7...",None
30829,None,100000063,10238,1,7.938151,1,None,None,connector,None,"LINESTRING (-77.3707 39.0143, -77.3707839 39.0...",None


In [18]:
reading = pd.read_csv(tmc_path + os.sep + 'Reading_VA.csv')
reading = reading[pd.to_datetime(reading['measurement_tstamp'], format='%Y-%m-%d %H:%M:%S')<datetime.datetime.strptime('2015-04-02', '%Y-%m-%d')]
reading

,_vatmc_code,measurement_tstamp,speed,reference_speed
0,110-05695,2015-04-01 00:00:00,31.0,31.0
1,110+05696,2015-04-01 00:00:00,32.0,32.0
2,110P05699,2015-04-01 00:00:00,36.0,36.0
3,110+05704,2015-04-01 00:00:00,50.0,50.0
4,110+05695,2015-04-01 00:00:00,31.0,31.0
...,...,...,...,...
16987,110+05710,2015-04-01 23:55:00,48.0,48.0
16988,110-05702,2015-04-01 23:55:00,48.0,48.0
16989,110N05694,2015-04-01 23:55:00,30.0,30.0
16990,110-05698,2015-04-01 23:55:00,30.0,30.0


In [ ]:
link_performance_csv = pd.DataFrame()
link_performance_csv['corridor_id'] = None
link_performance_csv['from_node_id'] = None
link_performance_csv['to_node_id'] = None
link_performance_csv['timestamp'] = None
link_performance_csv['volume'] = None
link_performance_csv['travel_time'] = None
link_performance_csv['speed'] = None
link_performance_csv['reference_speed'] = None
link_performance_csv['density'] = None
link_performance_csv['queue'] = None
link_performance_csv['notes'] = None


gp = reading.groupby('measurement_tstamp')
for key, form in gp:
    print(key)
    for i in link_csv.index:
        form_selected = form[form['_vatmc_code']==link_csv['name'][i]]
        if len(form_selected)>0:
        # break
            link_performance_csv = link_performance_csv.append({'corridor_id': link_csv['name'][i]+'_'+tmc.loc[tmc['tmc']==link_csv['name'][i],'direction'].values[0],\
                                            'from_node_id': link_csv.loc[i,'from_node_id'], \
                                            'to_node_id': link_csv.loc[i,'to_node_id'], \
                                            'timestamp': form_selected['measurement_tstamp'].values[0][0:10]+'T'+form_selected['measurement_tstamp'].values[0][11:13]+':'+form_selected['measurement_tstamp'].values[0][14:16],\
                                            'volume': None,\
                                            'travel_time': link_csv['length'][i]/form_selected['speed'].values[0],\
                                            'speed': form_selected['speed'].values[0],\
                                            'reference_speed': form_selected['reference_speed'].values[0],\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)
        else:
            link_performance_csv = link_performance_csv.append({'corridor_id': link_csv['name'][i]+'_'+tmc.loc[tmc['tmc']==link_csv['name'][i],'direction'].values[0],\
                                            'from_node_id': link_csv.loc[i,'from_node_id'], \
                                            'to_node_id': link_csv.loc[i,'to_node_id'], \
                                            'timestamp': None,\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed': None,\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)


link_performance_csv.to_csv(gmns_path + os.sep +'/link_performance_csv.csv')

In [ ]:
link_performance_csv